## Градиентный бустинг в лоб

In [1]:
import pandas as pd
import numpy as np

test = pd.read_csv('data/features_test.csv.zip', index_col='match_id')
train = pd.read_csv('data/features.csv.zip', index_col='match_id')
train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [2]:
[i for i, v in train.count().iteritems() if v < 97230]

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

In [3]:
train.fillna(0, inplace=True)
y = train['radiant_win']
X = train.drop(['start_time', 'duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire',
                           'barracks_status_radiant', 'barracks_status_dire'], axis=1)

test.fillna(0, inplace=True)
X_test = test.drop(['start_time'], axis=1)

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
import datetime
kfold = KFold(len(X), n_folds=5, shuffle=True, random_state=42)
for num_tree in [10,20,30,40,50]:
    clf = GradientBoostingClassifier(n_estimators=num_tree, random_state=42, verbose=False)
    start_time = datetime.datetime.now()
    score = cross_val_score(clf, X, y, cv=kfold, scoring='roc_auc')
    end_time = datetime.datetime.now()
    print("%s) time to fit = %s; ROC_AUC = %s\n" %
          (num_tree, end_time-start_time, score.mean()))

10) time to fit = 0:00:32.530852; ROC_AUC = 0.664850693272

20) time to fit = 0:01:08.568798; ROC_AUC = 0.682461881041

30) time to fit = 0:01:34.664953; ROC_AUC = 0.690006467861

40) time to fit = 0:01:56.963610; ROC_AUC = 0.693941559672

50) time to fit = 0:02:23.641244; ROC_AUC = 0.697493747258



In [6]:
clf = GradientBoostingClassifier(n_estimators=50, random_state=42, verbose=False)
start_time = datetime.datetime.now()
clf.fit(X, y)
end_time = datetime.datetime.now()
pred = clf.predict_proba(X_test)[:, 1]

In [9]:
print("Testing data) time to fit = %s;", end_time-start_time)
pred

Testing data) time to fit = %s; 0:00:38.724731


array([ 0.50416519,  0.6158742 ,  0.46393127, ...,  0.27854355,
        0.35118015,  0.51765001])

In [25]:
def hypothesis_filling():
    # Может быть заполнить пропуски по-другому? Всё-таки 0 - это номер возможного игрока и команды.
    # Лучше выбрать значения за пределами настоящих индексов.
    # Например, то что индексы положить -1.
    # То что время - сделать очень большим.
    train = pd.read_csv('data/features.csv', index_col='match_id')
    train['first_blood_team'].fillna(-1, inplace=True)
    train['first_blood_player1'].fillna(-1, inplace=True)
    train['first_blood_player2'].fillna(-1, inplace=True)
    train.fillna(100000, inplace=True)
    y = train['radiant_win']
    X = train.drop(['start_time', 'duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire',
                               'barracks_status_radiant', 'barracks_status_dire'], axis=1)
    kfold = KFold(len(X), n_folds=5, shuffle=True, random_state=42)
    for num_tree in [10,20,30]:
        clf = GradientBoostingClassifier(n_estimators=num_tree, random_state=42, verbose=False)
        start_time = datetime.datetime.now()
        score = cross_val_score(clf, X, y, cv=kfold, scoring='roc_auc')
        end_time = datetime.datetime.now()
        print("%s) time to fit = %s; ROC_AUC = %s\n" %
              (num_tree, end_time-start_time, score.mean()))

hypothesis_filling()

10) time to fit = 0:00:35.006071; ROC_AUC = 0.664825663953

20) time to fit = 0:01:03.569147; ROC_AUC = 0.68245927329

30) time to fit = 0:01:30.288316; ROC_AUC = 0.689702634742



## Логистическая регрессия

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

clf = LogisticRegression(random_state=42)
XX = StandardScaler().fit_transform(X)
yy = y.values
kfold = KFold(len(XX), n_folds=5, shuffle=True, random_state=42)
grid = {'C': np.linspace(0.001,1,10)}
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=kfold)
gs.fit(XX, y)

for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
print("Best param: ", gs.best_params_)

Quality 0.7156134126091879 for {'C': 0.001}
Quality 0.7158228841390879 for {'C': 0.112}
Quality 0.7158214065444368 for {'C': 0.223}
Quality 0.7158212677183129 for {'C': 0.33400000000000002}
Quality 0.7158210728598625 for {'C': 0.44500000000000001}
Quality 0.7158210039871449 for {'C': 0.55600000000000005}
Quality 0.7158208800403874 for {'C': 0.66700000000000004}
Quality 0.7158209531271614 for {'C': 0.77800000000000002}
Quality 0.7158210039619542 for {'C': 0.88900000000000001}
Quality 0.71582102408383 for {'C': 1.0}
Best param:  {'C': 0.112}
